In [ ]:
# read the blastn outfmt 6 file
blastn_file = '/data5/xzg_data/arabidopsis_single_cell_atalas/rna-seq/post-process/blastn_out.txt'

result_dict = {}

with open(blastn_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            continue
        line_list = line.split('\t')
        query_id = line_list[0]
        subject_id = line_list[1]
        percent_identity = float(line_list[2])
        alignment_length = int(line_list[3])
        mismatch = int(line_list[4])
        gap_open = int(line_list[5])
        q_start = int(line_list[6])
        q_end = int(line_list[7])
        s_start = int(line_list[8])
        s_end = int(line_list[9])
        evalue = float(line_list[10])
        bit_score = float(line_list[11])

        if query_id == subject_id:
            continue
        if query_id not in result_dict:
            result_dict[query_id] = []
        result_dict[query_id].append([subject_id, percent_identity, alignment_length, mismatch, gap_open, q_start, q_end, s_start, s_end, evalue, bit_score])

In [27]:
from trinity_post_process_v2 import read_salmon_sf, read_fasta, ORF, Gene, Isoform,  IsoformRank, filter_orf

In [28]:
# read trinity fasta
fasta_file = "/data6/xzg_data/trinity_ss_flnc.Trinity.fasta"

fasta_dict = read_fasta(fasta_file)


In [29]:
# read salmon output 
salmon_output_sf = [
    "/data6/xzg_data/salmon/L_quant.sf",
    "/data6/xzg_data/salmon/St_quant.sf",
    "/data6/xzg_data/salmon/Tip_quant.sf"
]


tpm_dict = read_salmon_sf(salmon_output_sf)

In [30]:
# Read and parse the BED file
bed_file = "/data6/xzg_data/transdecoer.standard.code/trinity_ss_flnc.Trinity.fasta.transdecoder.bed"

# tx : [orf1, orf2, orf3]
tx_orf_dict = {}

with open(bed_file, 'r') as file:
    # Skip the header
    next(file)
    
    # Parse each line
    for line in file:
        orf = ORF.parse_bed_line(line)
        transcript_name = orf.transcript_name
        if transcript_name not in tx_orf_dict:
            tx_orf_dict[transcript_name] = []
        tx_orf_dict[transcript_name].append(orf)


tx_orf_dict = filter_orf(tx_orf_dict, tpm_dict)


# 解析orf_dict, 得到isoform_dict
# gene: [tx1, tx2, tx3 ....]
gene_tx_dict = {}

for tx, orfs in tx_orf_dict.items():
    gene_name = "_".join(tx.split("_")[:-1])
    isoform = Isoform(tx, orfs)
    if gene_name not in gene_tx_dict:
        gene_tx_dict[gene_name] = []
    gene_tx_dict[gene_name].append(isoform)


# 解析isoform_dict, 得到gene_dict
gene_dict = {}
for gene_name, isoforms in gene_tx_dict.items():
    
    gene_dict[gene_name] = Gene(gene_name, isoforms)


In [36]:
len(tx_orf_dict.values())

78477

In [31]:
len(gene_dict)

38202

In [49]:
# 基于gene_dict筛选fasta_dict
from Bio import SeqIO
with open("/data6/xzg_data/trinity_ss_flnc.Trinity.fasta.filtered", 'w') as file:
    for gene_name, gene in gene_dict.items():
        for isoform in gene.isoforms:
            isoform_name = isoform.name
            SeqIO.write(fasta_dict[isoform_name], file, "fasta")


In [42]:
# 将基因按照isoform数目分成两类
single_isoform_gene = [ v for v in gene_dict.values() if v.size == 1]
multi_isoform_gene = [ v for v in gene_dict.values() if v.size > 1]

print(f"single isoform gene: {len(single_isoform_gene)}")
print(f"multi isoform gene: {len(multi_isoform_gene)}")

# 对于多个multi_isoform_gene的情况
good_gene_list = []
bad_gene_list = []
for gene in multi_isoform_gene:
    isoforms = gene.isoforms
    # 获取每个转录本的预测结果, 从中选取得分最高的
    highest_isoform = ""
    # 无穷小
    highest_score = float("-inf") 

    for isoform in isoforms:
        if isoform.score > highest_score:
            highest_score = isoform.score 
            highest_isoform = isoform
    
    if highest_isoform.rank == IsoformRank.single_complete:
        good_gene_list.append(gene)
    else:
        bad_gene_list.append(gene)

print(f"good gene: {len(good_gene_list)}")
print(f"bad gene: {len(bad_gene_list)}")

single isoform gene: 23079
multi isoform gene: 15123
good gene: 9263
bad gene: 5860
